# Perceptron

A Perceptron is an algorithm for supervised learning in binary classifier - it is a type of linear classifier.

TODO: expand

## Basic Implementation

In [3]:
import numpy as np

class Perceptron(object):
    """
    
    """
    
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter
    
    def fit(self, X, y):
        """
        Fit training data
        
        Params:
        --------
        eta : float
            Learning rate (0.0 - 1.0)
        
        n_iter : int
            Passes over the training dataset
            
        
        Attributes:
        ---------
        
        w_ : 1d-array / vector
            Weights after fitting
            
        errors_ : list
            Number of misclassifications in every epoch
            
        
        Additional information:
        ---------
        * An epoch in machine learning is the number passes over the training set        
        
        """
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0,0, -1)
    

## Let's Grab Data

In [4]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.tail()

,0,1,2,3,4
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


## Data Analysis